### Yoly Network

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from tqdm import tqdm
from cv2.ximgproc import guidedFilter
from skimage.color import rgb2hsv
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

import numpy as np
import sys

sys.path.insert(0, '..')
from utils.collection import HazeCollection
from utils.network import VaeNet, Net
from utils.loss import StdLoss
from utils.utils import get_atmosphere, torch_to_np, np_to_torch, save_image, get_information
from utils.imresize import np_imresize

#### Collection

In [2]:
haze_path_dt1 = "/home/maldonadoq/Datasets/Reside/Standard/HSTS/synthetic/synthetic"
image_path_dt1 = "/home/maldonadoq/Datasets/Reside/Standard/HSTS/synthetic/original"

haze_path_dt2 = "/home/maldonadoq/Datasets/Reside/Standard/HSTS/real-world"

haze_path_dt3 = "/home/maldonadoq/Datasets/Reside/Standard/SOTS/indoor/hazy_val"
image_path_dt3 = "/home/maldonadoq/Datasets/Reside/Standard/SOTS/indoor/gt_val"

In [3]:
size = 256
channels = 3

collection_dt1 = HazeCollection(haze_path_dt1, image_path_dt1)
collection_dt2 = HazeCollection(haze_path_dt2)
collection_dt3 = HazeCollection(haze_path_dt3, image_path_dt3)

#### Model

In [4]:
# UNet
un = Net(channels)

x = torch.zeros([1, channels, size, size])
a = un(x)
a.shape

torch.Size([1, 3, 256, 256])

#### Training

In [5]:
class YolyTrainer:
    def __init__(
        self,
        metrics=[]
    ):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.metrics = metrics

    def init(self, image):
        self.model = {
            "netJ": Net(out_channel=3).to(self.device),
            "netT": Net(out_channel=1).to(self.device),
            "netA": VaeNet(image.shape).to(self.device)
        }
        self.loss = {
            "mse": nn.MSELoss().to(self.device),
            "std": StdLoss().to(self.device)
        }

        atmosphere = get_atmosphere(image)
        self.valA = nn.Parameter(data=torch.cuda.FloatTensor(
            atmosphere.reshape((1, 3, 1, 1))), requires_grad=False)

        parameters = [p for p in self.model["netJ"].parameters()]
        parameters += [p for p in self.model["netT"].parameters()]
        parameters += [p for p in self.model["netA"].parameters()]

        self.optimizer = optim.Adam(
            parameters,
            lr=self.learning_rate
        )

    def t_matting(self, mask_out_np, original):
        refine_t = guidedFilter(original.transpose(1, 2, 0).astype(
            np.float32), mask_out_np[0].astype(np.float32), 50, 1e-4)
        return np.array([np.clip(refine_t, 0.1, 1)])

    def train(self, information, epochs):
        (haze, image, name) = information
        self.init(haze)
        loop = tqdm(range(epochs))

        losses = []
        lossFinal = None

        hazeTorch = np_to_torch(haze)
        hazeTorch = hazeTorch.to(device=self.device)
        for i in loop:
            # forward
            predT = self.model["netT"](hazeTorch)
            predJ = self.model["netJ"](hazeTorch)
            predA = self.model["netA"](hazeTorch)

            #lossT = self.loss["std"](predT)
            lossJ = self.loss["mse"](predT * predJ + (1 - predT) * predA, hazeTorch)
            lossA = self.model["netA"].getLoss()

            hsv = np_to_torch(rgb2hsv(torch_to_np(predJ).transpose(1, 2, 0)))
            cap_prior = hsv[:, :, :, 2] - hsv[:, :, :, 1]
            cap_loss = self.loss["mse"](cap_prior, torch.zeros_like(cap_prior))

            lossFinal = lossJ + lossA
            lossFinal += 1.0 * cap_loss
            lossFinal += 0.1 * self.loss["std"](predA)
            lossFinal += self.loss["mse"](predA,self.valA * torch.ones_like(predA))

            #if i % 100 == 0 or i == epochs-1:
            if i == epochs-1:
                imageJ = np.clip(torch_to_np(predJ), 0, 1)
                imageT = np.clip(torch_to_np(predT), 0, 1)
                imageA = np.clip(torch_to_np(predA), 0, 1)
                imageT = self.t_matting(imageT, haze)

                # I(x) = J(x)t(x) + A(1 − t(x))
                post = np.clip((haze - ((1 - imageT) * imageA)) / imageT, 0, 1)
                post = np_imresize(post, output_shape=haze.shape[1:])
                save_image('{}_{}'.format(name[0].split('.')[0], i+1), post, '../images/yoly/' + self.dt_number)

            # backward
            self.optimizer.zero_grad()
            lossFinal.backward(retain_graph=True)
            self.optimizer.step()

            # update tqdm
            loop.set_postfix(loss=lossFinal.item())
            losses.append(lossFinal.item())
        
        # metrics
        finalSSIM, finalPSNR = 0, 0
        if image is not None:
            finalSSIM = ssim(post, image, channel_axis=0, data_range=1)
            finalPSNR = psnr(post, image)
            print("Ssim: {:.4f}, Psnr: {:.4f}\n".format(finalSSIM, finalPSNR))

        return [np.sum(losses)/len(losses), finalSSIM, finalPSNR]

    def fit(
        self,
        collection,
        epochs=1,
        dt_number='',
        learning_rate=1e-3
    ):
        historial = []
        self.dt_number = dt_number
        self.learning_rate = learning_rate
        #loop = enumerate(tqdm(collection))
        for i, info in enumerate(collection):
            print('Image {}/{} [{}]'.format(i+1, len(collection), info[2][0]))
            loss, ssim, psnr = self.train(info, epochs)
            historial.append([loss, ssim, psnr])
        return historial

In [6]:
dehazing = YolyTrainer()

#### Testing

In [7]:
epochs = 512
learning_rate = 1e-4

##### D1

In [8]:
historial_dt1 = dehazing.fit(collection_dt1, epochs, 'd1/', learning_rate)
get_information(historial_dt1, collection_dt1)

Image 1/10 [1381.jpg]


100%|██████████| 512/512 [00:25<00:00, 20.22it/s, loss=0.179]


Ssim: 0.8976, Psnr: 22.0904

Image 2/10 [5576.jpg]


100%|██████████| 512/512 [00:18<00:00, 27.23it/s, loss=0.213]


Ssim: 0.8276, Psnr: 21.9937

Image 3/10 [7471.jpg]


...

Ssim: 0.9306, Psnr: 22.6941

Image 9/10 [1352.jpg]


100%|██████████| 512/512 [00:46<00:00, 10.92it/s, loss=0.425]


Ssim: 0.9457, Psnr: 23.5561

Image 10/10 [4561.jpg]


100%|██████████| 512/512 [00:19<00:00, 26.06it/s, loss=0.266]

Ssim: 0.6719, Psnr: 18.9987

Mean SSIM: 0.8713289260864258
Mean PSNR: 22.590697622819587
Best SSIM: 1352.jpg
Best PSNR: 3146.jpg


##### D2

In [9]:
historial_dt2 = dehazing.fit(collection_dt2, epochs, 'd2/', learning_rate)
get_information(historial_dt2, collection_dt2)

Image 1/10 [SFC_Google_197.jpeg]


100%|██████████| 512/512 [00:36<00:00, 14.13it/s, loss=0.152]


Image 2/10 [MLS_Bing_117.jpeg]


100%|██████████| 512/512 [00:40<00:00, 12.49it/s, loss=0.353]


Image 3/10 [HazyDr_Google_396.jpeg]


...

Image 9/10 [KRO_Google_143.jpeg]


100%|██████████| 512/512 [00:41<00:00, 12.45it/s, loss=0.347]


Image 10/10 [HazeDr_Google_404.jpeg]


100%|██████████| 512/512 [01:43<00:00,  4.97it/s, loss=0.312]

Mean SSIM: 0.0
Mean PSNR: 0.0
Best SSIM: SFC_Google_197.jpeg
Best PSNR: SFC_Google_197.jpeg


##### D3

In [11]:
historial_dt3 = dehazing.fit(collection_dt3, epochs, 'd3/', learning_rate)
get_information(historial_dt3, collection_dt3)

Image 1/10 [1410_5.png]


100%|██████████| 512/512 [00:30<00:00, 16.57it/s, loss=0.159]


Ssim: 0.6715, Psnr: 16.0638

Image 2/10 [1430_5.png]


100%|██████████| 512/512 [00:31<00:00, 16.08it/s, loss=0.0705]


Ssim: 0.5940, Psnr: 15.3952

Image 3/10 [1440_5.png]


...

Ssim: 0.8528, Psnr: 17.0706

Image 9/10 [1425_5.png]


100%|██████████| 512/512 [00:32<00:00, 15.92it/s, loss=0.298]


Ssim: 0.7583, Psnr: 15.4174

Image 10/10 [1420_5.png]


100%|██████████| 512/512 [00:31<00:00, 16.17it/s, loss=0.342]

Ssim: 0.8625, Psnr: 19.6535

Mean SSIM: 0.7694881558418274
Mean PSNR: 17.49236738200319
Best SSIM: 1415_5.png
Best PSNR: 1415_5.png
